<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 1

Run date time (IST): 2023-07-25 17:18:54


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=9fd3e7cd316affac16db0ae48e08fab32f8426f646f90ab2d8db40bc5a07a9ab
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

myts = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myTransactionsReport.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'BAJAJFINSV',
       'BAJAJHIND', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP', 'CAMS',
       'CDSL', 'CLEAN', 'DMART', 'EICHERMOT', 'EMAMILTD', 'FINEORG',
       'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC', 'HDFCBANK', 'ICICIGI',
       'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITBEES', 'J&KBANK',
       'JMFINANCIL', 'JUBLFOOD', 'KOTAKBANK', 'LTIM', 'LUXIND',
       'METROPOLIS', 'NIACL', 'NMDC', 'PAGEIND', 'PIDILITIND', 'PNB',
       'RAJESHEXPO', 'RELAXO', 'ROSSARI', 'SAIL', 'SRF', 'STARHEALTH',
       'SUNTV', 'TCS', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL',
       'VALIANTORG', 'WIPRO'], dtype=object)

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='7y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    tmp['Target'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [7]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

df_prec_dev["Target"] = df_prec_dev["Target"].fillna(df_prec_dev["Max"])

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Target'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)
print('-------------------')
print('Total profit: ',round(sum(tmp_df[tmp_df['Current P/L%'] > 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] > 0]['Investment']),0))
print('Total loss: ',round(sum(tmp_df[tmp_df['Current P/L%'] < 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] < 0]['Investment']),0))

qualified stocks: 52
Investment:  3443229.0
Current:  3625593.0
Today PnL:  12765.0
Today PnL %:  0.35
Current PnL:  182364.0
Current PnL %:  5.3
Estimate:  6105003.0
Estimate PnL:  2661774.0
Estimate PnL %:  77.3
-------------------
Total profit:  222251.0
Total loss:  -39887.0


In [20]:
cols = ['Close', 'Stock', 'Max', 'Target', 'Avg_Cost', 'Shares', 'Criteria', 'Strategy', 'Today P/L%', 'Current P/L%', 'Estimated P/L%', 'Juice Left%', 'Weightage%', 'Risk Ind']
tmp_df = tmp_df[cols]

In [21]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Target,Avg_Cost,Shares,Criteria,Strategy,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
0,453.65,5PAISA.NS,553.0,553.0,347.06,228.0,V40N,ATH,15.24,30.71,59.34,28.63,2.30,71.0
19,2527.05,HDFCAMC.NS,3518.0,3518.0,1973.85,50.0,V40,ATH,1.18,28.03,78.23,50.20,2.87,80.0
17,5511.50,GILLETTE.NS,7529.0,7529.0,4475.85,21.0,V40,ATH,0.07,23.14,68.21,45.07,2.73,63.0
48,44.00,UJJIVANSFB.NS,57.0,57.0,36.90,1850.0,V40N,ATH,3.17,19.24,54.47,35.23,1.98,38.0
21,1373.10,ICICIGI.NS,1613.0,1613.0,1177.58,84.0,V40,ATH,-1.04,16.60,36.98,20.38,2.87,48.0
16,202.10,GICRE.NS,358.0,358.0,174.18,231.0,VR,2T3Y,1.84,16.03,105.53,89.50,1.17,19.0
45,538.40,SUNTV.NS,710.0,710.0,473.49,108.0,V40N,ATH,-0.54,13.71,49.95,36.24,1.49,20.0
4,1648.85,BAJAJFINSV.NS,1905.0,1905.0,1454.93,52.0,V40,ATH,-0.52,13.33,30.93,17.60,2.20,29.0
7,1701.00,BATAINDIA.NS,2173.0,2173.0,1523.62,100.0,V40,ATH,0.93,11.64,42.62,30.98,4.42,51.0
1,1619.25,AAVAS.NS,3196.0,3196.0,1453.92,37.0,VR,2T3Y,0.98,11.37,119.82,108.45,1.56,18.0


In [22]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Target,Avg_Cost,Shares,Criteria,Strategy,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
37,2607.40,PIDILITIND.NS,2902.0,2902.0,2504.06,30.0,V40,ATH,-1.67,4.13,15.89,11.76,2.18,9.0
46,3399.15,TCS.NS,3847.0,3847.0,3231.00,16.0,V40,ATH,0.13,5.20,19.07,13.87,1.50,8.0
13,3326.20,EICHERMOT.NS,3850.0,3850.0,3260.20,15.0,V40N,ATH,0.55,2.02,18.09,16.07,1.42,3.0
4,1648.85,BAJAJFINSV.NS,1905.0,1905.0,1454.93,52.0,V40,ATH,-0.52,13.33,30.93,17.60,2.20,29.0
30,1875.50,KOTAKBANK.NS,2210.0,2270.0,1955.95,26.0,V40,BTT,-1.15,-4.11,16.06,20.17,1.48,-6.0
21,1373.10,ICICIGI.NS,1613.0,1613.0,1177.58,84.0,V40,ATH,-1.04,16.60,36.98,20.38,2.87,48.0
20,1696.60,HDFCBANK.NS,1728.0,2050.0,1683.55,30.0,V40,BTT,1.08,0.78,21.77,20.99,1.47,1.0
26,31.32,ITBEES.NS,40.0,40.0,31.49,3350.0,AR,ATH,-0.29,-0.54,27.02,27.56,3.06,-2.0
0,453.65,5PAISA.NS,553.0,553.0,347.06,228.0,V40N,ATH,15.24,30.71,59.34,28.63,2.30,71.0
43,2164.10,SRF.NS,2821.0,2821.0,2222.00,14.0,VR,ATH,0.92,-2.61,26.96,29.57,0.90,-2.0


In [23]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Target,Avg_Cost,Shares,Criteria,Strategy,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1227.15,GLAND.NS,4316.0,4316.0,1225.87,55.0,VR,2T3Y,-0.98,0.10,252.08,251.98,1.96,0.0
50,506.40,VALIANTORG.NS,1851.0,1851.0,532.61,185.0,VR,2T3Y,-1.51,-4.92,247.53,252.45,2.86,-14.0
32,1491.50,LUXIND.NS,4546.0,4546.0,1493.72,51.0,V40N,2T3Y,0.70,-0.15,204.34,204.49,2.21,-0.0
49,341.00,VAIBHAVGBL.NS,966.0,966.0,317.48,144.0,VR,2T3Y,-0.35,7.41,204.27,196.86,1.33,10.0
6,218.20,BANDHANBNK.NS,723.0,723.0,244.52,400.0,VR,2T3Y,-0.21,-10.76,195.68,206.44,2.84,-31.0
33,1403.30,METROPOLIS.NS,3440.0,3440.0,1365.98,22.0,VR,ATH,-0.32,2.73,151.83,149.10,0.87,2.0
34,123.75,NIACL.NS,287.0,287.0,117.07,390.0,VR,ATH,0.32,5.71,145.15,139.44,1.33,8.0
47,2413.20,TEAMLEASE.NS,5281.0,5281.0,2279.93,21.0,V40N,2T3Y,0.92,5.85,131.63,125.78,1.39,8.0
22,1501.65,INDIGOPNTS.NS,3112.0,3112.0,1407.73,124.0,V40N,2T3Y,0.57,6.67,121.07,114.40,5.07,34.0
1,1619.25,AAVAS.NS,3196.0,3196.0,1453.92,37.0,VR,2T3Y,0.98,11.37,119.82,108.45,1.56,18.0


In [24]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Target,Avg_Cost,Shares,Criteria,Strategy,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
22,1501.65,INDIGOPNTS.NS,3112.0,3112.0,1407.73,124.0,V40N,2T3Y,0.57,6.67,121.07,114.40,5.07,34.0
36,37196.20,PAGEIND.NS,53549.0,53549.0,38548.77,4.0,V40,ATH,-0.70,-3.51,38.91,42.42,4.48,-16.0
7,1701.00,BATAINDIA.NS,2173.0,2173.0,1523.62,100.0,V40,ATH,0.93,11.64,42.62,30.98,4.42,51.0
24,1334.60,INFY.NS,1848.0,1848.0,1324.07,104.0,V40,ATH,-0.15,0.80,39.57,38.77,4.00,3.0
26,31.32,ITBEES.NS,40.0,40.0,31.49,3350.0,AR,ATH,-0.29,-0.54,27.02,27.56,3.06,-2.0
21,1373.10,ICICIGI.NS,1613.0,1613.0,1177.58,84.0,V40,ATH,-1.04,16.60,36.98,20.38,2.87,48.0
19,2527.05,HDFCAMC.NS,3518.0,3518.0,1973.85,50.0,V40,ATH,1.18,28.03,78.23,50.20,2.87,80.0
50,506.40,VALIANTORG.NS,1851.0,1851.0,532.61,185.0,VR,2T3Y,-1.51,-4.92,247.53,252.45,2.86,-14.0
6,218.20,BANDHANBNK.NS,723.0,723.0,244.52,400.0,VR,2T3Y,-0.21,-10.76,195.68,206.44,2.84,-31.0
17,5511.50,GILLETTE.NS,7529.0,7529.0,4475.85,21.0,V40,ATH,0.07,23.14,68.21,45.07,2.73,63.0


In [25]:
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Target,Avg_Cost,Shares,Criteria,Strategy,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
6,218.20,BANDHANBNK.NS,723.0,723.0,244.52,400.0,VR,2T3Y,-0.21,-10.76,195.68,206.44,2.84,-31.0
39,513.20,RAJESHEXPO.NS,949.0,949.0,578.92,87.0,V40N,ATH,-0.42,-11.35,63.93,75.28,1.46,-17.0
36,37196.20,PAGEIND.NS,53549.0,53549.0,38548.77,4.0,V40,ATH,-0.70,-3.51,38.91,42.42,4.48,-16.0
50,506.40,VALIANTORG.NS,1851.0,1851.0,532.61,185.0,VR,2T3Y,-1.51,-4.92,247.53,252.45,2.86,-14.0
11,1297.10,CLEAN.NS,2648.0,2648.0,1365.11,39.0,VR,2T3Y,-1.02,-4.98,93.98,98.96,1.55,-8.0
35,112.20,NMDC.NS,178.0,178.0,122.55,265.0,VR,ATH,1.08,-8.45,45.25,53.70,0.94,-8.0
30,1875.50,KOTAKBANK.NS,2210.0,2270.0,1955.95,26.0,V40,BTT,-1.15,-4.11,16.06,20.17,1.48,-6.0
25,619.45,IRCTC.NS,1164.0,1164.0,633.05,120.0,AR,ATH,-0.14,-2.15,83.87,86.02,2.21,-5.0
27,67.85,J&KBANK.NS,74.0,107.0,70.75,425.0,VR,ATH,-4.30,-4.10,51.24,55.34,0.87,-4.0
31,4868.65,LTIM.NS,7380.0,7380.0,4987.59,10.0,V200,ATH,-0.21,-2.38,47.97,50.35,1.45,-3.0


In [29]:
# SIP based on portfolio
tmp_df[tmp_df['Weightage%'] < 1].sort_values(by = 'Current P/L%', ascending=True)

,Close,Stock,Max,Target,Avg_Cost,Shares,Criteria,Strategy,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
35,112.20,NMDC.NS,178.0,178.0,122.55,265.0,VR,ATH,1.08,-8.45,45.25,53.70,0.94,-8.0
27,67.85,J&KBANK.NS,74.0,107.0,70.75,425.0,VR,ATH,-4.30,-4.10,51.24,55.34,0.87,-4.0
43,2164.10,SRF.NS,2821.0,2821.0,2222.00,14.0,VR,ATH,0.92,-2.61,26.96,29.57,0.90,-2.0
14,423.55,EMAMILTD.NS,588.0,588.0,422.59,72.0,V200,ATH,0.67,0.23,39.14,38.91,0.88,0.0
9,2336.85,CAMS.NS,3867.0,3867.0,2324.85,13.0,VR,ATH,0.04,0.52,66.33,65.81,0.88,0.0
42,92.25,SAIL.NS,130.0,130.0,90.90,330.0,VR,ATH,2.61,1.49,43.01,41.52,0.87,1.0
33,1403.30,METROPOLIS.NS,3440.0,3440.0,1365.98,22.0,VR,ATH,-0.32,2.73,151.83,149.10,0.87,2.0
41,879.25,ROSSARI.NS,1569.0,1569.0,844.40,36.0,VR,2T3Y,-0.54,4.13,85.81,81.68,0.88,4.0
44,642.45,STARHEALTH.NS,907.0,907.0,599.50,50.0,VR,ATH,-0.86,7.16,51.29,44.13,0.87,6.0
5,18.75,BAJAJHIND.NS,24.0,24.0,17.15,1750.0,VR,ATH,-2.34,9.33,39.94,30.61,0.87,8.0


In [28]:
#myts['Value'] = myts['Value'].str.replace(',', '').astype(float)

AttributeError: ignored

In [ ]:
#sum(myts[(myts['Type'] == 'Buy')]['Value'])

In [ ]:
#sum(myts[(myts['Type'] == 'Sell')]['Value'])